In [5]:
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict

In [11]:
class BioFaceNet(nn.Module):
    def __init__(self):
        super(BioFaceNet, self).__init__()
        self.skip = []
        inputChannels = 3
        nfeatures = np.array([32,64,128,256,512])
        kernelSize = 3
        pad = 1
        nclasses = 4
        LightVectorSize = 15
        bSize = 2
        dims = LightVectorSize+bSize
        
        ################################# Encoding #####################################
        self.conv1dn = nn.Conv2d(inputChannels,nfeatures[0], kernel_size = 3,padding = pad,bias=False)
        self.bnorm1 = nn.BatchNorm2d(num_features = nfeatures[0])
        self.relu = nn.ReLU(inplace=True)
        self.conv2dn = nn.Conv2d(nfeatures[0],nfeatures[0], kernel_size = (3,3),padding = pad,bias=False)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv3dn = nn.Conv2d(nfeatures[0],nfeatures[1], kernel_size = (3,3),padding = pad,bias=False)
        self.bnorm2 = nn.BatchNorm2d(num_features = nfeatures[1])
        self.conv4dn = nn.Conv2d(nfeatures[1],nfeatures[1], kernel_size = (3,3),padding = pad,bias=False)
        
        self.conv5dn = nn.Conv2d(nfeatures[1],nfeatures[2], kernel_size = (3,3),padding = pad,bias=False)
        self.bnorm3 = nn.BatchNorm2d(num_features = nfeatures[2])
        self.conv6dn = nn.Conv2d(nfeatures[2],nfeatures[2], kernel_size = (3,3),padding = pad,bias=False)
        
        self.conv7dn = nn.Conv2d(nfeatures[2],nfeatures[3], kernel_size = (3,3),padding = pad,bias=False)
        self.bnorm4 = nn.BatchNorm2d(num_features = nfeatures[3])
        self.conv8dn = nn.Conv2d(nfeatures[3],nfeatures[3], kernel_size = (3,3),padding = pad,bias=False)
        
        self.conv9dn = nn.Conv2d(nfeatures[3],nfeatures[4], kernel_size = (3,3),padding = pad,bias=False)
        self.bnorm5 = nn.BatchNorm2d(num_features = nfeatures[4])
        self.conv10dn = nn.Conv2d(nfeatures[4],nfeatures[4], kernel_size = (3,3),padding = pad,bias=False)
        ##############################################################################
        
        ################################# Decoding #####################################
        self.upsample1 = nn.Upsample(size = (27,22))
        self.upsample2 = nn.Upsample(size = (54,44))
        self.upsample3 = nn.Upsample(size = (109,89))
        self.upsample4 = nn.Upsample(size = (218,178))
        self.upsamp = nn.Upsample(scale_factor=2, mode='bilinear')
        
        self.conv1up = nn.Conv2d(nfeatures[4]+nfeatures[3],nfeatures[3], kernel_size=(3,3),padding = pad,bias=False)
        self.conv2up = nn.Conv2d(nfeatures[3]+nfeatures[2],nfeatures[2], kernel_size=(3,3),padding = pad,bias=False)
        self.conv3up = nn.Conv2d(nfeatures[2]+nfeatures[1],nfeatures[1], kernel_size=(3,3),padding = pad,bias=False)
        self.conv4up = nn.Conv2d(nfeatures[1]+nfeatures[0],nfeatures[0], kernel_size=(3,3),padding = pad,bias=False)
        self.convFD = nn.Conv2d(nfeatures[0],1, kernel_size= (3,3),padding = pad,bias=False)
        ##############################################################################
        
        ################################# FCN #####################################
        self.convFCN1 = nn.Conv2d(in_channels=nfeatures[4], out_channels=nfeatures[4], kernel_size = 4)
        self.bnormFCN1 = nn.BatchNorm2d(num_features=nfeatures[4])
        self.reluFCN1 = nn.ReLU(inplace=True)
        self.convFCN2 = nn.Conv2d(in_channels=nfeatures[4], out_channels=nfeatures[4], kernel_size = 1)
        self.bnormFCN2 = nn.BatchNorm2d(num_features=nfeatures[4])
        self.reluFCN2 = nn.ReLU(inplace=True)
        ##############################################################################
        
        self.pred = nn.Conv2d(in_channels = nfeatures[4],out_channels = 17, kernel_size = 2,bias=False)
        self.predLinear = nn.Linear(in_features = 17,out_features=17)
        
        ####scaling####
        
    def forward(self, x):
        #Assuming that the doubleconv will always be true
        
        x = x.permute(0,3,1,2) # only use this when using original imgData
        ########Encoding################
        x = self.conv1dn(x) #iteration 1
        #print("shape: "+str(x.shape))
        x = self.bnorm1(x)
        #print("shape: "+str(x.shape))
        x = self.relu(x)
        #print("shape: "+str(x.shape))
        x = self.conv2dn(x)
        #print("shape: "+str(x.shape))
        x = self.bnorm1(x)
        #print("shape: "+str(x.shape))
        x = self.relu(x)
        #print("shape: "+str(x.shape))
        x = self.conv2dn(x)
        #print("shape: "+str(x.shape))
        x = self.bnorm1(x)
        #print("shape: "+str(x.shape))
        encode1 = self.relu(x)
        #print("Eshape: "+str(encode1.shape))
        #self.skip.append(x)
        x= self.pool(encode1)
        #print("shape: "+str(x.shape))
        
        x = self.conv3dn(x) #iteration 2
        #print("shape: "+str(x.shape))
        x = self.bnorm2(x)
        #print("shape: "+str(x.shape))
        x = self.relu(x)
        #print("shape: "+str(x.shape))
        x = self.conv4dn(x)
        #print("shape: "+str(x.shape))
        x = self.bnorm2(x)
        #print("shape: "+str(x.shape))
        x = self.relu(x)
        #print("shape: "+str(x.shape))
        x = self.conv4dn(x)
        #print("shape: "+str(x.shape))
        x = self.bnorm2(x)
        #print("shape: "+str(x.shape))
        encode2 = self.relu(x)
        #print("Eshape: "+str(encode2.shape))
        #self.skip.append(x)
        x =self.pool(encode2)
        #print("shape: "+str(x.shape))
        
        x = self.conv5dn(x) #iteration 3
        #print("shape: "+str(x.shape))
        x = self.bnorm3(x)
        #print("shape: "+str(x.shape))
        x = self.relu(x)
        #print("shape: "+str(x.shape))
        x = self.conv6dn(x)
        #print("shape: "+str(x.shape))
        x = self.bnorm3(x)
        #print("shape: "+str(x.shape))
        x = self.relu(x)
        #print("shape: "+str(x.shape))
        x = self.conv6dn(x)
        #print("shape: "+str(x.shape))
        x = self.bnorm3(x)
        #print("shape: "+str(x.shape))
        encode3 = self.relu(x)
        #print("Eshape: "+str(encode3.shape))
        #self.skip.append(x)
        x =self.pool(encode3)
        #print("shape: "+str(x.shape))
        
        x = self.conv7dn(x) #iteration 4
        #print("shape: "+str(x.shape))
        x = self.bnorm4(x)
        #print("shape: "+str(x.shape))
        x = self.relu(x)
        #print("shape: "+str(x.shape))
        x = self.conv8dn(x)
        #print("shape: "+str(x.shape))
        x = self.bnorm4(x)
        #print("shape: "+str(x.shape))
        x = self.relu(x)
        #print("shape: "+str(x.shape))
        x = self.conv8dn(x)
        #print("shape: "+str(x.shape))
        x = self.bnorm4(x)
        #print("shape: "+str(x.shape))
        encode4 = self.relu(x)
        #print("Eshape: "+str(encode4.shape))
        #self.skip.append(x)
        x =self.pool(encode4)
        #print("shape: "+str(x.shape))
        
        x = self.conv9dn(x) #iteration 5
        #print("shape: "+str(x.shape))
        x = self.bnorm5(x)
        #print("shape: "+str(x.shape))
        x = self.relu(x)
        #print("shape: "+str(x.shape))
        x = self.conv10dn(x)
        #print("shape: "+str(x.shape))
        x = self.bnorm5(x)
        #print("shape: "+str(x.shape))
        x = self.relu(x)
        #print("shape: "+str(x.shape))
        x = self.conv10dn(x)
        #print("shape: "+str(x.shape))
        x = self.bnorm5(x)
        #print("shape: "+str(x.shape))
        encode5 = self.relu(x)
        #print("Eshape: "+str(encode5.shape))
        #self.skip.append(x)
        y =self.pool(encode5)
        #print("shape: "+str(y.shape))
        ################################
        
        
        
        ########Decoding################
        for c in range(1,5):
            x = self.upsample1(y) ### iteration 1
            #print("----------------Decoding----------------")
            #print("X shape: "+str(x.shape))
            #print("Encode5 shape: "+str(encode5.shape))
            x = torch.cat((x,encode4),1)
            x = self.conv1up(x)
            x = self.bnorm4(x)
            x = self.relu(x)
            x = self.conv8dn(x) #not a down conv., this has the same param for a double conv.
            x = self.bnorm4(x)
            x = self.relu(x)
            x = self.conv8dn(x) #not a down conv., this has the same param for a double conv.
            x = self.bnorm4(x)
            x = self.relu(x)
            
            x = self.upsample2(x) ### iteration 2
            x = torch.cat((x,encode3),1)
            x = self.conv2up(x)
            x = self.bnorm3(x)
            x = self.relu(x)
            x = self.conv6dn(x) #not a down conv., this has the same param for a double conv.
            x = self.bnorm3(x)
            x = self.relu(x)
            x = self.conv6dn(x) #not a down conv., this has the same param for a double conv.
            x = self.bnorm3(x)
            x = self.relu(x)
            
            x = self.upsample3(x) ### iteration 3
            x = torch.cat((x,encode2),1)
            x = self.conv3up(x)
            x = self.bnorm2(x)
            x = self.relu(x)
            x = self.conv4dn(x) #not a down conv., this has the same param for a double conv.
            x = self.bnorm2(x)
            x = self.relu(x)
            x = self.conv4dn(x) #not a down conv., this has the same param for a double conv.
            x = self.bnorm2(x)
            x = self.relu(x)

            x = self.upsample4(x) ### iteration 4
            x = torch.cat((x,encode1),1)
            x = self.conv4up(x)
            x = self.bnorm1(x)
            x = self.relu(x)
            x = self.conv2dn(x) #not a down conv., this has the same param for a double conv.
            x = self.bnorm1(x)
            x = self.relu(x)
            x = self.conv2dn(x) #not a down conv., this has the same param for a double conv.
            x = self.bnorm1(x)
            x = self.relu(x)
            
            x = self.convFD(x)
            
            if c==1:
                z = x
            else:
                z = torch.cat((z,x),1)
        x = z
        ################################
        
        ######## FCN ################
        fcn1 = self.convFCN1(y)
        fcn1 = self.bnormFCN1(fcn1)
        fcn1 = self.reluFCN1(fcn1)
        fcn2 = self.convFCN2(fcn1)
        fcn2 = self.bnormFCN2(fcn2)
        fcn2 = self.reluFCN2(fcn2)
        #############################
        
        predictions = self.pred(fcn2)
        #predictions = self.predLinear(predictions)
        
        
        #print("prediction shape: ",predictions.shape)
        lightingParameters = predictions[:,0:LightVectorSize,0,0]
        batch = predictions[:,15:17,0,0]
        #print(batch.shape)
        #nbatch = batch.shape[0]*batch.shape[2]*batch.shape[3]
        b = batch
        fmel = x[:,0,:,:]
        fblood = x[:,1,:,:]
        shading = x[:,2,:,:]
        specmask = x[:,3,:,:]

        return lightingParameters,b,fmel,fblood,shading,specmask
    

class ScalingNet(nn.Module):
    def __init__(self):
        super(ScalingNet, self).__init__()
        self.skip = []
        inputChannels = 3
        nfeatures = np.array([32,64,128,256,512])
        kernelSize = 3
        pad = 1
        nclasses = 4
        LightVectorSize = 15
        bSize = 2
        dims = LightVectorSize+bSize
        
                ####scaling####
        self.soft = nn.Softmax()
        self.sig = nn.Sigmoid()
        
    def forward(lightingParameters,b,fmel,fblood,shading,specmask):
        numBatch = 17
        lightweights = self.softmax(lightingparameters[:,0:15,:,:])
        weightA = lightingWeights[:,0,:,:,] 
        weightD = lightingWeights[:,1,:,:,]
        Fweights = lightingWeights[:,2:14,:,:,]
        CCT =  lightingparameters[:,:,14,:]
        CCT = ((22-1) / (1 + torch.exp(-1*CCT))) + 1
        b = 6*self.sig(b) - 3
        Bgrid = b.reshape(2,1,1,numBatch)
        Bgrid = Bgrid/3
        fmel = 2*self.sig(fmel) - 1
        fblood = 2*self.sig(fmel) - 1
        shading = torch.exp(shading)
        specmask = torch.exp(shading)
        return weightA,weightD,CCT,Fweights,b,BGrid,fmel,fblood,Shading,specmask